In [ ]:
#!pip install scikit-learn 
import os
import copy
import pandas as pd
import numpy as np
import matplotlib 
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import sklearn
import mne
from mne.time_frequency import psd_welch
import torch
import csv
import datetime
from datetime import timedelta
datetimeFormat = '%Y-%m-%d %H:%M:%S.%f'
import re

from numpy.fft import fft, fftfreq
from scipy import signal
from mne.time_frequency import tfr_morlet

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
%matplotlib notebook
mne.set_log_level('ERROR') 

path_to_data = './data_files.txt'
path_to_csv = './Invasive_data.csv'

data_dict = {}

def square_data(inputs):
    data_df = pd.Series(inputs)
    data_df_sq = data_df.pow(2)
    data_df_np = data_df_sq.to_numpy()
    return data_df_np

def normalize_data(inputs):
    data_df_HRC1 = pd.Series(inputs)
    data_df_std_dev = data_df_HRC1.rolling(time_steps).std() # 30 seconds
    data_df_norm = data_df_HRC1.divide(data_df_std_dev)  
    data_df_norm[:time_steps-1].update(data_df_HRC1[:time_steps-1]) # if NaN, norm = original value
    data_df_HRC1_np = data_df_norm.to_numpy()
    return data_df_HRC1_np

def diff_data(inputs):  #TODO : Do this before rereferencing
    data_df = pd.Series(inputs)
    data_df_diff= data_df.diff()
    data_df_np = data_df_diff.to_numpy()
    return data_df_np

def dur(num):
    if num/60 > 1:
        return 60.0
    else:
        return float(num)

# Set reference channel for all data points for all channels individually
HRA = ['HRA1','HRA2','HRA3','HRA4','HRA5']
HRB = ['HRB1','HRB2','HRB3','HRB4','HRB5','HRC1','HRC2','HRC3','HRC4','HRC5']
HRC = ['HRB1','HRB2','HRB3','HRB4','HRB5','HRC1','HRC2','HRC3','HRC4','HRC5']

if torch.cuda.is_available():
    print('CUDA-enabled GPU found. Training should be faster.')
else:
    print('No GPU found. Training will be slow')
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'

my_database ={'Patient 1':{'PatientID': 0, 'FileID': [], 'Frequency': 512, 'Duration': 0}}
num_of_files = range(8)

#fnames = fetch_data(subjects=subjects, recording=recordings, on_missing='warn') # LIST
print('HERE WE GO!!!!!!!!!!!!!!')
with open(path_to_data) as file:
    lines = file.readlines()
    lines_iter = iter(lines)
    lines = [line.rstrip() for line in lines]
    print('lines', lines, len(lines))
            
def load_raw_dataset(line, my_database):
    """Load a recording from the iEEG dataset.
    
    Parameters
    ----------
    raw_fname : str
        Path to the .data file containing the raw data.
    
    Returns
    -------
    mne.io.Raw :
        Raw object containing the EEG. 
    """
    #for line in range(0,len(lines)):
    print(line, type(line))
    #if not line.startswith("#"):
    entry = 'EEG_inv_' + str(line)[-18:-15] + '_' + str(line)[-9:-5]
    print('Value for entry',entry)
    my_database['Patient 1']['PatientID'] = str(line)[-18:-15]
    my_database['Patient 1']['FileID'] = str(line)[-9:-5]
    #print(my_database)
    data_raw =  mne.io.read_raw_nicolet(str(line), 'eeg', preload=True)
    
    return data_raw

# Check 
#load_raw_dataset(lines,my_database)
# Load recordings
data_raws = [load_raw_dataset(line, my_database) for line in lines]
print('Database entries',len(data_raws) , type(data_raws) )
#print(data_raws[0].info)
#print(data_raws[0].plot())

In [ ]:
#Generate spike using 
    #1 Rate coding: spikegen.rate   (On normal data and on first derivvative of data)
    #2 Latency coding: spikegen.latency
    #3 Delta modulation: spikegen.delta
#Delta Encoding

#!pip install plotly
import snntorch as snn
from snntorch import spikegen
import matplotlib.pyplot as plt
import snntorch.spikeplot as splt
from IPython.display import HTML
dtype = torch.float
from matplotlib.widgets import Slider,Button
import plotly.express as px
from snntorch import surrogate
from snntorch import backprop
from snntorch import functional as SF
from snntorch import utils
from snntorch import spikeplot as splt

import torch
import torch.nn as nn
import tensorflow as tf
import torch.nn.functional as F
import itertools

In [1]:
def process_file(data_raw, csv_row ):
    df = pd.read_csv(path_to_csv, usecols=['meas_date','onset', 'offset', 'channels'])
    cnt_data =0
    cnt_data_ch = 0
    columns_ = []
    data_ch_dict = [None] * len(data_raw)
    data_epoch = [None] * len(data_raw)
    
    for col in range(0,len(df.columns)):
        data = df.iloc[csv_row,col]
        columns_.append(data)
    
    meas_data = columns_[0]
    onset = columns_[1]
    offset = columns_[2]
    channel = columns_[3]

    tdur_seiz = datetime.datetime.strptime(offset, datetimeFormat)- datetime.datetime.strptime(onset, datetimeFormat)
    tdiff_on = datetime.datetime.strptime(onset, datetimeFormat)- datetime.datetime.strptime(meas_data, datetimeFormat)
    tdiff_off = datetime.datetime.strptime(offset, datetimeFormat)- datetime.datetime.strptime(meas_data, datetimeFormat)
    #print("seizure starts after:",tdiff_on, "seizure lasts for",tdur_seiz,"stops after",tdiff_off)
    #print('Seizure onset',onset,'Seizure offset',offset )
    #print(print("seizure starts after:",tdiff_on.seconds, "seizure lasts for",tdur_seiz.seconds,"stops after",tdiff_off.seconds))

    for arr in range(len(channel)):
        for lis in range(len(channel[arr])):
            channel[arr][lis].replace('\'','')
            channel[arr][lis].replace(']','')
            channel[arr][lis].replace('[','')
            channel[arr][lis].replace(' ','')
    #print("Shape and type of tdiff_on:",type(tdiff_on),tdiff_on, 'channel',channel)
    
    if channel.find(',') != -1:
        channel = list(channel.split(","))
        #rint('HEY FOUND A ,', channel,len(channel), type(channel))
    else:
        channel = list(channel.split(","))
        #print('HEY DID NOT FIND ,', channel, len(channel), type(channel))
    
    cnt_data += 1

    #ch_name = entry 
    ref_ch = HRB

    data_ch_dict = data_raw.copy()
    #print('Before re-referencing',data_ch_dict.info)
    #data_ch_dict.plot()
    data_ch_dict = data_ch_dict.pick_channels(ch_names= ref_ch, ordered=False)
    data_ch_dict.set_eeg_reference(ref_channels=ref_ch)
    #print('After re-referencing',data_ch_dict.info)
    #data_ch_dict.plot()

    data_ch_dict.load_data().filter(l_freq=0.2, h_freq=48, method='iir')
    #print('After filtering',data_ch_dict.info)

    do_normalization = False
    calculate_dt= False
    #time_steps = (10*60)/2
    time_steps = 10
    
    #Calculate time difference     #before re-referencing TODO
    if calculate_dt == True:
        #print('First derivative', channel)
        data_ch_dict.apply_function(diff_data, picks=channel)
        #print('After first derivative',data_ch_dict.info)
        #data_ch_dict.plot()
    else:
        data_ch_dict = data_ch_dict
        #print('First derivative not taken')
    
    # TODO ; Temporary use HRB2 and HRC2
    channel_temp = ['HRB2','HRC2']
    data_ch_dict.pick_channels(ch_names = channel_temp, ordered = False)
    #data_ch_dict.plot()

    #Crop the data to include less non-ictal activity : Temporary logic
    def define_df(data_ch_dict, idx):
        df1 = data_ch_dict.to_data_frame()
        df1['time_s'] = df1['time'] / 1000.0
        df1['target'] = 0
        df1['target'] = np.where(( (df1.time_s+ idx >= np.float(tdiff_on.seconds)) & (df1.time_s + idx <= np.float(tdiff_off.seconds))),1,df1.target)
        return df1
    
    if (tdiff_on.seconds < 1200):
        data_ch_dict_n = data_ch_dict.copy().crop(tmin=0,tmax=250) #0,1200
        #print('Seizure occurs in first half')
        df_subset = define_df(data_ch_dict_n, 0)
    elif (tdiff_on.seconds > 2400):
        data_ch_dict_n = data_ch_dict.copy().crop(tmin=2500, tmax =2800) # 2400,3600
        print('Seizure occurs in end')
        df_subset = define_df(data_ch_dict_n, 2500)
    else:
        data_ch_dict_n = data_ch_dict.copy().crop(tmin=1200,tmax=2400) # 1200,2400
        print('Seizure occurs in mid ')
        df_subset = define_df(data_ch_dict_n, 1300)
        
    #data_ch_dict_n.plot()
    
    '''print(df_subset)
    # Store data, time and label as dataframe
    df = data_ch_dict.to_data_frame()
    df['time_s'] = df['time'] / 1000.0
    df['target'] = 0
    df['target'] = np.where(( (df.time_s >= np.float(tdiff_on.seconds)) & (df.time_s <= np.float(tdiff_off.seconds))),1,df.target)
    #print(df)'''
    
    #Extract 5s epochs : Create epochs and labels
    interval = 5  #TODO 2 sec
    time_window = interval * data_ch_dict.info['sfreq'] # 5*512
    
    data_epoch = mne.make_fixed_length_epochs(data_ch_dict_n, duration= interval, preload=True, id= csv_row) 
    #print('Fixed Length Epochs',csv_row,data_epoch, len(data_epoch), type(data_epoch) )
    #print('Epochs return value',data_epoch.get_data(), np.shape(data_epoch.get_data()), type(data_epoch.get_data()) )
    
    data_labels = [None] * len(data_epoch)  # 720
    for seg in range(0,len(data_epoch)):
        df_start = int(seg*time_window)
        df_end = int((seg+1)*time_window)
        #print('df start and end', df_start, df_end, len(data_epoch),df_subset.loc[df_start:df_end])
        count_labels = df_subset.loc[df_start:df_end,'target'].sum()
        if count_labels > 1000:
            #print('SEIZURE DETECTED', csv_row, ':', count_labels,'seg', seg, 'start',df_start,'end',df_end)
            label = 1
        else:
            #print('NO SEIZURE DETECTED FOR', csv_row,':',count_labels,'seg', seg, 'start',df_start,'end',df_end)
            label = 0
        data_labels[seg] = label
        count_labels = 0
    #print( np.shape(np.array(data_labels)),type(np.array(data_labels)) )   #RETURN
    #print('Label generated for',csv_row,':',data_epoch[0])
    
    '''#Visualize label generation
    print(df_subset)
    t1 = df_subset.time_s
    t2 = df_subset.time_s

    plt.figure()
    plt.subplot(211)
    plt.plot(t1, df_subset.HRB2)

    plt.subplot(212)
    plt.plot(t2, df_subset.target)
    plt.show()#'''
    return data_epoch.get_data() , np.array(data_labels)

In [ ]:
from torch.utils.data import Dataset, ConcatDataset
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor

class EpochsDataset(Dataset):
    """Class to expose an MNE Epochs object as PyTorch dataset.
    
    Parameters
    ----------
    epochs_data : np.ndarray
        The epochs data, shape (n_epochs, n_channels, n_times).
    epochs_labels : np.ndarray
        The epochs labels, shape (n_epochs,)
    transform : callable | None
        The function to eventually apply to each epoch
        for preprocessing (e.g. scaling). Defaults to None.
    """
    def __init__(self, epochs_data, epochs_labels, transform=None):
        assert len(epochs_data) == len(epochs_labels)
        self.epochs_data = epochs_data
        self.epochs_labels = epochs_labels
        self.transform = transform

    def __len__(self):
        return len(self.epochs_labels)

    def __getitem__(self, idx):
        X, y = self.epochs_data[idx], self.epochs_labels[idx]
        if self.transform is not None:
            X = self.transform(X)
        X = torch.as_tensor(X[None, ...])
        return X, y

#transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#with dt Mean [[ 2.3293406e-22][-2.8587362e-22]     Std Dev [[2.29891510e-05][1.07505107e-05]]
#without dt Mean [[-1.35525272e-21] [ 6.77626358e-22]] Std Dev [[6.45315196e-05] [3.75374791e-05]]
def scale(X):
    """Standard scaling of data along the last dimention.
    
    Parameters
    ----------
    X : array, shape (n_channels, n_times)
        The input signals.
        
    Returns
    -------
    X_t : array, shape (n_channels, n_times)
        The scaled signals.
    """
    X -= np.mean(X, axis=1, keepdims=True)
    #print('Mean',np.mean(X, axis=1, keepdims=True))
    #print('Std Dev',  np.std(X, axis=1, keepdims=True))
    return X / np.std(X, axis=1, keepdims=True)

transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))])

# Apply windowing and move to pytorch dataset
all_datasets = [EpochsDataset(*process_file(data_raw, csv_row), transform=scale) 
                for csv_row,data_raw in enumerate(data_raws)]

# Concatenate into a single dataset
dataset = ConcatDataset(all_datasets)

In [ ]:
# Making training test validation split
from sklearn.model_selection import LeavePGroupsOut  #TODO; Do LOO split for validation

def train_test_split(dataset):
    """Split dataset into train and test 
    
    Parameters
    ----------
    dataset : ConcatDataset
        The dataset to split.
        
    Returns
    -------
    ConcatDataset
        The training data and the testing data.
    """
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    print('Train and test size', train_size, test_size)
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    return train_dataset, test_dataset

torch.manual_seed(87)
np.random.seed(87)
train_ds, test_ds = train_test_split(dataset)
print('Number of examples in each set:')
print(f'Training: {len(train_ds)}')
print(f'Test: {len(test_ds)}')
print(train_ds)

## Computing class weight
from sklearn.utils.class_weight import compute_class_weight

classes_mapping = {0: 'non-ictal', 1: 'ictal'}
train_y = pd.Series([y for _, y in train_ds]).map(classes_mapping)
class_weights = compute_class_weight('balanced', classes=np.unique(train_y), y=train_y)
print(class_weights)
#Visualizing the class distribution, Very imbalanced right now #TODO
#ax_ = train_y.value_counts().plot(kind='barh')
#ax_.set_xlabel('Number of training examples');
#ax_.set_ylabel('Sleep stage');

In [ ]:
# TRAIN AND MONITOR NETWORK
from torch.utils.data import DataLoader

# Create dataloaders
train_batch_size = 47  # Important hyperparameter For Now : (720/4)
valid_batch_size = 94  # Can be made as large as what fits in memory; won't impact performance
num_workers = 0  # Number of processes to use for the data loading process; 0 is the main Python process

train_loader = DataLoader(
    train_ds, batch_size=train_batch_size, shuffle=True, num_workers=num_workers)
#loader_valid = DataLoader(
#    valid_ds, batch_size=valid_batch_size, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(
    test_ds, batch_size=valid_batch_size, shuffle=False, num_workers=num_workers)
print('Training files in a batch', len(train_loader),' Testing files in a batch', len(test_loader))
print('Training files total ', len(train_ds),' Testing files total', len(test_ds))

In [ ]:
#a basic supervised learning algorithm will be implemented.
#train a multi-layer fully-connected spiking neural network using gradient descent to perform image classification.
############### TUTORIAL 5 ########################
import torch.nn as nn
import torch.nn.functional as F
from snntorch import surrogate

# Network Architecture
num_inputs = 2*2560
num_mid1 = 1024
num_mid2 = 512
num_mid3 = 256
num_outputs = 2

# Temporal Dynamics
num_steps = 25
beta = 0.95
device = 'cpu'
batch_size = 47  # Change to 180 later / 126
#spike_grad = surrogate.fast_sigmoid(slope=25)

# Define Network
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # Initialize layers
        self.fc1 = nn.Linear(num_inputs, num_mid1)
        self.lif1 = snn.Leaky(beta=beta)
        self.fc2 = nn.Linear(num_mid1, num_outputs)
        self.lif2 = snn.Leaky(beta=beta)
        #self.fc3 = nn.Linear(num_mid2, num_mid3)
        #self.lif3 = snn.Leaky(beta=beta)
        #self.fc4 = nn.Linear(num_mid3, num_outputs)
        #self.lif4 = snn.Leaky(beta=beta)

    def forward(self, x):

        # Initialize hidden states at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        #mem3 = self.lif3.init_leaky()
        #mem4 = self.lif4.init_leaky()
        
        # Record the final layer
        spk2_rec = []
        mem2_rec = []

        for step in range(num_steps):
            #print('x',x.size())
            cur1 = self.fc1(x)
            #print('cur1',cur1.size())
            spk1, mem1 = self.lif1(cur1, mem1)
            #print('spk1',spk1.size(), 'mem1',mem1.size())
            cur2 = self.fc2(spk1)
            #print('cur2',cur2.size())
            spk2, mem2 = self.lif2(cur2, mem2)
            '''
            #print('spk2',spk2.size(), 'mem2',mem2.size())
            cur3 = self.fc3(spk2)
            #print('cur3',cur3.size())
            spk3, mem3 = self.lif3(cur3, mem3)
            #print('spk3',spk3.size(), 'mem3',mem3.size())
            cur4 = self.fc4(spk3)
            #print('cur4',cur4.size())
            spk4, mem4 = self.lif4(cur4, mem4)
            #print('spk4',spk4.size(), 'mem4',mem4.size())'''
            spk2_rec.append(spk2)
            mem2_rec.append(mem2)
        #print('output', torch.stack(spk2_rec, dim=0).size, torch.stack(spk2_rec).size()) 
        return torch.stack(spk2_rec, dim=0), torch.stack(mem2_rec, dim=0)

# Load the network onto CUDA if available
net = Net().to(device)

# pass data into the network, sum the spikes over time
# and compare the neuron with the highest number of spikes
# with the target

def print_batch_accuracy(data, targets, train=False):
    output, _ = net(data.view(batch_size, -1))
    check_all_output = output.sum(dim=0)
    print('spk_rec values', check_all_output.size(),'zero',check_all_output[0],'one', check_all_output[1],'last',check_all_output[46] )
    val, idx = output.sum(dim=0).max(1)
    print('idx from',output.sum(dim=0).max(1), 'idx.size',idx.size(), 'targets', targets)
    acc = np.mean((targets == idx).detach().cpu().numpy())
    print('acc', acc)

    if train:
        print(f"Train set accuracy for a single minibatch: {acc*100:.2f}%")
    else:
        print(f"Test set accuracy for a single minibatch: {acc*100:.2f}%")

def train_printer():
    print(f"Epoch {epoch}")
    print(f"Train Set Loss: {loss_hist[counter]:.2f}")
    print(f"Test Set Loss: {test_loss_hist[counter]:.2f}")
    print_batch_accuracy(data, targets, train=True)
    print_batch_accuracy(test_data, test_targets, train=False)
    print("\n")
    
#loss = nn.CrossEntropyLoss()
#loss = nn.CrossEntropyLoss(weight=torch.Tensor(class_weights).to(device)) 
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4, betas=(0.9, 0.999))
loss = SF.ce_rate_loss()

In [ ]:
#@title Plotting Settings
def plot_cur_mem_spk(cur, mem, spk, thr_line=False, vline=False, title=False, ylim_max1=2.5, ylim_max2=2.5, num_steps =200):
    # Generate Plots
    fig, ax = plt.subplots(3, figsize=(8,6), sharex=True, 
                        gridspec_kw = {'height_ratios': [1, 1, 0.4]})

    # Plot input current
    ax[0].plot(cur, c="tab:orange")
    ax[0].set_ylim([-ylim_max1, ylim_max1])
    ax[0].set_xlim([0, num_steps])
    ax[0].set_ylabel("Input Current ($I_{in}$)")
    if title:
        ax[0].set_title(title)

    # Plot membrane potential
    ax[1].plot(mem)
    ax[1].set_ylim([-ylim_max2, ylim_max2]) 
    ax[1].set_ylabel("Membrane Potential ($U_{mem}$)")
    if thr_line:
        ax[1].axhline(y=thr_line, alpha=0.25, linestyle="dashed", c="black", linewidth=2)
    plt.xlabel("Time step")

    # Plot output spike using spikeplot
    splt.raster(spk, ax[2], s=400, c="black", marker="|") # s is size of scatter points
    if vline:
        ax[2].axvline(x=vline, ymin=0, ymax=6.75, alpha = 0.15, linestyle="dashed", c="black", linewidth=2, zorder=0, clip_on=False)
    plt.ylabel("Output spikes")
    plt.yticks([]) 

    plt.show()

def plot_snn_spikes(spk_in, spk1_rec, spk2_rec, title):
    # Generate Plots
    fig, ax = plt.subplots(3, figsize=(8,7), sharex=True, 
                        gridspec_kw = {'height_ratios': [1, 1, 0.4]})

    # Plot input spikes
    splt.raster(spk_in[:,0], ax[0], s=0.03, c="black")
    ax[0].set_ylabel("Input Spikes")
    ax[0].set_title(title)

    # Plot hidden layer spikes
    splt.raster(spk1_rec.reshape(num_steps, -1), ax[1], s = 0.05, c="black")
    ax[1].set_ylabel("Hidden Layer")

    # Plot output spikes
    splt.raster(spk2_rec.reshape(num_steps, -1), ax[2], c="black", marker="|")
    ax[2].set_ylabel("Output Spikes")
    ax[2].set_ylim([0, 1])

    plt.show()

In [ ]:
######   Tutorial 3: A Feedforward Spiking Neural Network
num_steps = 2560

data, targets = next(iter(train_loader))   # Normalized data
data = data.to(device=device, dtype=torch.float32)
#print('data',data.size(), data)
#data = spikegen.delta(data, threshold=0.5)
targets = targets.to(device=device, dtype=torch.int64)
print('targets',targets.size(), targets)

In [ ]:
#Continued Plot spike for 1 datapoint of channel when target =1
lif1 = snn.Leaky(beta=0.819, threshold=1.2)

def leaky_integrate_and_fire(mem, x, w, beta, threshold=1.2):
    spk = (mem > threshold) # if membrane exceeds threshold, spk=1, else, 0
    mem = beta * mem + w*x - spk*threshold
    return spk, mem

data_point = 2
x = data[data_point][0][0]
cur_in = data[data_point][0][0]
print('x is for channel 1',x.size(), x, 'with the target',targets[data_point] )
mem = torch.zeros(1)
spk_out = torch.zeros(1)
mem_rec = []
spk_rec = []

# neuron parameters
w = 0.5
beta = 0.819

# neuron simulation
for step in range(num_steps):
    #spk, mem = leaky_integrate_and_fire(mem, x[step], w=w, beta=beta) # Replace with w*X[t], mem
    spk, mem = lif1(cur_in[step], mem)
    mem_rec.append(mem)
    spk_rec.append(spk)

# convert lists to tensors
mem_rec = torch.stack(mem_rec)
spk_rec = torch.stack(spk_rec)

plot_cur_mem_spk(x*w, mem_rec, spk_rec, thr_line=1.2,ylim_max1=2.5,
                 title="LIF Neuron Model With Weighted Step Voltage", num_steps=num_steps)

In [ ]:
#Plot spike for 1 datapoint of channel when target =0
data_point = 1
x = data[data_point][0][0]
cur_in = data[data_point][0][0]
print('x is for channel 1',x.size(), x, 'with the target',targets[data_point] )
mem = torch.zeros(1)
spk_out = torch.zeros(1)
mem_rec = []
spk_rec = []

# neuron parameters
w = 0.5
beta = 0.819

# neuron simulation
for step in range(num_steps):
    #spk, mem = leaky_integrate_and_fire(mem, x[step], w=w, beta=beta)
    spk, mem = lif1(cur_in[step], mem)
    mem_rec.append(mem)
    spk_rec.append(spk)

# convert lists to tensors
mem_rec = torch.stack(mem_rec)
spk_rec = torch.stack(spk_rec)

plot_cur_mem_spk(x*w, mem_rec, spk_rec, thr_line=1.2,ylim_max1=2.5,
                 title="LIF Neuron Model With Weighted Step Voltage", num_steps=num_steps)

In [ ]:
#Feedforward Spiking Neural Network
# layer parameters
num_inputs = 2560   # What should be the correct input
num_hidden = 1000
num_outputs = 2
beta = 0.819
data_point = 2   # Chosen a batch with truth =1

# initialize layers
fc1 = nn.Linear(num_inputs, num_hidden)
lif1 = snn.Leaky(beta=0.819, threshold=0.5)  #1.2
fc2 = nn.Linear(num_hidden, num_outputs)
lif2 = snn.Leaky(beta=0.819, threshold=0.5)

# Initialize hidden states
mem1 = lif1.init_leaky()
mem2 = lif2.init_leaky()

# record outputs
mem2_rec = []
spk1_rec = []
spk2_rec = []

'''The number of time steps will be set to that of the longest recording in the mini-batch, 
   and all other samples will be padded with zeros to match it.'''
#Dimension of input should be for 1 batch time_step X 1X  feature_dimension (spk_in)
#spk_in = spikegen.rate_conv(torch.rand((200, 784))).unsqueeze(1) #Should be 2560 X 1 X 1
spk_in = ((data[data_point][0][0]).repeat(2560,1)).unsqueeze(1)   #([2560, 1, 2560])
#spk_in = spikegen.delta(spk_in_o, threshold=0.01)
print(f"Dimensions of spk_in: {spk_in.size()}")

# network simulation
for step in range(num_steps):
    cur1 = fc1(spk_in[step]) # post-synaptic current <-- spk_in x weight
    spk1, mem1 = lif1(cur1, mem1) # mem[t+1] <--post-syn current + decayed membrane
    cur2 = fc2(spk1)
    spk2, mem2 = lif2(cur2, mem2)

    mem2_rec.append(mem2)
    spk1_rec.append(spk1)
    spk2_rec.append(spk2)

# convert lists to tensors
mem2_rec = torch.stack(mem2_rec)
spk1_rec = torch.stack(spk1_rec)
spk2_rec = torch.stack(spk2_rec)
print('mem_rec', mem2_rec.shape, 'spk1', spk1_rec.shape, 'spk2', spk2_rec.shape)
print('spk_in_o',spk_in, 'mem_rec',mem2_rec[0][0][:], 'spk1', spk1_rec[0][0][:], 'spk2', spk2_rec)

#plot_snn_spikes(spk_in, spk1_rec, spk2_rec, "Fully Connected Spiking Neural Network")

In [ ]:
from IPython.display import HTML

fig, ax = plt.subplots(facecolor='w', figsize=(12, 7))
labels=['non-ictal', 'ictal']
spk2_rec = spk2_rec.squeeze(1).detach().cpu()

#  Plot spike count histogram
anim = splt.spike_count(spk2_rec, fig, ax, labels=labels, animate=True)
HTML(anim.to_html5_video())

In [ ]:
##### TUtorial 5 : Training Spiking CNNs ################### for 1 channel
import torch.nn as nn
import torch.nn.functional as F
from snntorch import surrogate

# Network Architecture
num_inputs = 2560
num_mid1 = 1000
#num_mid2 = 512
#num_mid3 = 256
num_outputs = 2

# Temporal Dynamics
num_steps = 2560
beta = 0.819
device = 'cpu'
batch_size_train = 47  # Change to 180 later / 126
batch_size_test = 94 
#spike_grad = surrogate.fast_sigmoid(slope=25)

# Define Network
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # Initialize layers
        self.fc1 = nn.Linear(num_inputs, num_mid1)
        self.lif1 = snn.Leaky(beta=beta, threshold=0.1)
        self.fc2 = nn.Linear(num_mid1, num_outputs)
        self.lif2 = snn.Leaky(beta=beta, threshold=0.1)
        #self.fc3 = nn.Linear(num_mid2, num_mid3)
        #self.lif3 = snn.Leaky(beta=beta)
        #self.fc4 = nn.Linear(num_mid3, num_outputs)
        #self.lif4 = snn.Leaky(beta=beta)

    def forward(self, x):

        # Initialize hidden states at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        #mem3 = self.lif3.init_leaky()
        #mem4 = self.lif4.init_leaky()
        
        # Record the final layer
        spk2_rec = []
        mem2_rec = []

        for step in range(num_steps):
            #print('x',x.size())
            cur1 = self.fc1(x)
            #print('cur1',cur1.size())
            spk1, mem1 = self.lif1(cur1, mem1)
            #print('spk1',spk1.size(), 'mem1',mem1.size())
            cur2 = self.fc2(spk1)
            #print('cur2',cur2.size())
            spk2, mem2 = self.lif2(cur2, mem2)
            '''
            #print('spk2',spk2.size(), 'mem2',mem2.size())
            cur3 = self.fc3(spk2)
            #print('cur3',cur3.size())
            spk3, mem3 = self.lif3(cur3, mem3)
            #print('spk3',spk3.size(), 'mem3',mem3.size())
            cur4 = self.fc4(spk3)
            #print('cur4',cur4.size())
            spk4, mem4 = self.lif4(cur4, mem4)
            #print('spk4',spk4.size(), 'mem4',mem4.size())'''
            spk2_rec.append(spk2)
            mem2_rec.append(mem2)
        #print('output', torch.stack(spk2_rec, dim=0).size, torch.stack(spk2_rec).size()) 
        return torch.stack(spk2_rec, dim=0), torch.stack(mem2_rec, dim=0)

# Load the network onto CUDA if available
net = Net().to(device)

# pass data into the network, sum the spikes over time
# and compare the neuron with the highest number of spikes
# with the target

def print_batch_accuracy(data, targets,batch_size=batch_size_test ,train=False):
    output, _ = net(data.view(batch_size, -1))
    check_all_output = output.sum(dim=0)
    print('spk_rec values', check_all_output.size(),'Spike accumulated for both labels', check_all_output )
    val, idx = output.sum(dim=0).max(1)
    print('idx from',output.sum(dim=0).max(1), 'idx.size',idx.size(), 'targets', targets)
    acc = np.mean((targets == idx).detach().cpu().numpy())
    #print('acc', acc)

    if train:
        print(f"Train set accuracy for a single minibatch: {acc*100:.2f}%")
        acc_hist.append(acc.item())
    else:
        print(f"Test set accuracy for a single minibatch: {acc*100:.2f}%")
        test_acc_hist.append(acc.item())

def train_printer():
    print(f"Epoch {epoch}")
    print('Counter', counter)
    print(f"Train Set Loss: {loss_hist[counter]:.2f}")
    print(f"Test Set Loss: {test_loss_hist[counter]:.2f}")
    print_batch_accuracy(data, targets, batch_size=batch_size_train, train=True)
    print_batch_accuracy(test_data, test_targets, batch_size=batch_size_test,train=False)
    print(f"Train Accuracy: {acc_hist[counter]:.2f}")
    print(f"Test Accuracy: {test_acc_hist[counter]:.2f}")
    print("\n")
    
#loss = nn.CrossEntropyLoss()
#loss = nn.CrossEntropyLoss(weight=torch.Tensor(class_weights).to(device)) 
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4, betas=(0.9, 0.999))
#loss = SF.ce_rate_loss()
loss = nn.CrossEntropyLoss()

In [ ]:
num_epochs = 10
loss_hist = []
test_loss_hist = []
acc_hist = []
test_acc_hist = []
counter = 0
indices = torch.tensor([0])

# Outer training loop
for epoch in range(num_epochs):
    iter_counter = 0
    
    # Minibatch training loop
    for data, targets in iter(train_loader):
        data = data.to(device=device, dtype=torch.float32)
        data = torch.index_select(data, 2, indices)
        #rint('data', data.shape, data)
        check_nan = torch.isnan(data)
        #print('idx for nan in training',tf.where(check_nan))
        print('does input have nan training', torch.count_nonzero(check_nan))
        if (torch.count_nonzero(check_nan) != 0):
            corrupted_batch = (tf.where(check_nan))[0][0]
            #print('corrupted data', data.shape,'corrupt batch',corrupted_batch)
            for i in range(0,47):
                print('check if data is corrupt',i,data[i])
        #print('data',data.size(), data)
        #data = spikegen.delta(data, threshold=0.5)
        #print('spikegen data',data.size(), data)
        targets = targets.to(device=device, dtype=torch.int64)
        #print('targets',targets.size(), targets)

        # forward pass
        net.train()
        #print('data',data.shape, 'input data shape', data.view(batch_size_train, -1).shape)
        spk_rec, mem_rec = net(data.view(batch_size_train, -1))
        print('######## DURING TRAINING ############')
        #print('spk_rec',spk_rec.size(), 'mem_rec',mem_rec.size())
        #print('spk_rec',spk_rec, 'mem_rec',mem_rec)
        for name, param in net.named_parameters():
            if param.requires_grad:
                print('Name of parameter',name, 'Parameter data', (param.data).shape,param.data)
        print('mem_rec',mem_rec)
        #print('Non-zero elements in spk_rec',torch.count_nonzero(spk_rec))
        
        # initialize the loss & sum over time
        loss_val = torch.zeros((1), dtype=dtype, device=device)
        for step in range(num_steps):
            #print('Check loss dims', num_steps, mem_rec[step].shape, targets.shape)
            loss_val += loss(mem_rec[step], targets)
        #print('Loss val for train ', loss_val)
        #loss_val = loss(spk_rec, targets)

        # Gradient calculation + weight update
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

        # Store loss history for future plotting
        loss_hist.append(loss_val.item())
        
        #train_printer()
        
        # Test set
        with torch.no_grad():
            net.eval()
            test_data, test_targets = next(iter(test_loader))
            test_data = test_data.to(device=device, dtype=torch.float32)
            test_data = torch.index_select(test_data, 2, indices)
            print('test_data', test_data.shape, test_data)
            check_nan = torch.isnan(test_data)
            print('idx for nan in testing',tf.where(check_nan))
            print('does input have nan testing', torch.count_nonzero(check_nan))
            #test_data = spikegen.delta(test_data, threshold=0.5)
            test_targets = test_targets.to(device=device, dtype=torch.int64)

            # Test set forward pass
            #print('test_data',test_data.shape, 'input test_data', test_data.view(batch_size_test, -1).shape)
            test_spk, test_mem = net(test_data.view(batch_size_test, -1))
            print('######## DURING TESTING ############')
            #print('spk_rec',test_spk.size(), 'mem_rec',test_mem.size())
            #print('spk_rec',test_spk, 'mem_rec',test_mem)
            for name, param in net.named_parameters():
                if param.requires_grad:
                    print('Name of parameter',name, 'Parameter data', (param.data).shape,param.data)
            print('mem_rec',test_mem)
            
            # Test set loss
            test_loss = torch.zeros((1), dtype=dtype, device=device)
            for step in range(num_steps):
                test_loss += loss(test_mem[step], test_targets)
            #print('Loss val for test ', test_loss)
            test_loss_hist.append(test_loss.item())

            # Print train/test loss/accuracy
            #if counter % 10 == 0:
            train_printer()
            counter += 1
            #iter_counter +=1
            #print('counter', counter, 'iter_counter', iter_counter)
        print('####### END OF AN ITERATION')
        
''' # For one sample

data, targets = next(iter(train_loader))   # Normalized data
data = data.to(device=device, dtype=torch.float32)
indices = torch.tensor([0])
data = torch.index_select(data, 2, indices)                      # Select only 1 channel
#data = spikegen.delta(data, threshold=0.5)
targets = targets.to(device=device, dtype=torch.int64)
print('batch_size', data.shape)
spk_rec, mem_rec = net(data.view(batch_size, -1))
print(mem_rec.size())

# initialize the total loss value
loss_val = torch.zeros((1), dtype=dtype, device=device)
loss_val = loss(spk_rec, targets)

print('loss_val', loss_val, loss_val.size())
print(f"Training loss: {loss_val.item():.3f}")

print_batch_accuracy(data, targets, train=True)

# clear previously stored gradients
optimizer.zero_grad()

# calculate the gradients
loss_val.backward()

# weight update
optimizer.step()

# calculate new network outputs using the same data
spk_rec, mem_rec = net(data.view(batch_size, -1))

# initialize the total loss value
loss_val = torch.zeros((1), dtype=dtype, device=device)
    
loss_val = loss(spk_rec, targets)

print(f"Training loss: {loss_val.item():.3f}")
print_batch_accuracy(data, targets, train=True)''' # end of one sample

'''
        _, idx = spk_rec.sum(dim=0).max(1)
        print('spk_rec values and indices',spk_rec.sum(dim=0).max(1), 'idx:', idx , 'idx.size',idx.size())
        acc = np.mean((targets == idx).detach().cpu().numpy())
        print('acc', acc)
        
        loss_val = torch.zeros((1), dtype=dtype, device=device)

        # sum loss at every step
        for step in range(num_steps):
            loss_val += loss(mem_rec[step], targets)

        print(f"Training loss: {loss_val.item():.3f}")
        
        # clear previously stored gradients
        optimizer.zero_grad()

        # calculate the gradients
        loss_val.backward()

        # weight update
        optimizer.step()
        
        # calculate new network outputs using the same data
        spk_rec, mem_rec = net(data.view(batch_size, -1))
        print('spk_rec 2',spk_rec.size(), 'mem_rec',mem_rec.size())
        print('spk_rec 2',spk_rec, 'mem_rec',mem_rec)
        print('Non-zero elements in spk_rec II',torch.count_nonzero(spk_rec))
        
        _, idx = spk_rec.sum(dim=0).max(1)  # Returns the value and column index at which maximum value is stored
        print('spk_rec values and indices II',spk_rec.sum(dim=0).max(1), 'idx:', idx , 'idx.size',idx.size())
        
        
        # initialize the total loss value
        #loss_val = torch.zeros((1), dtype=dtype, device=device)

        # sum loss at every step
        #for step in range(num_steps):
        #    loss_val += loss(mem_rec[step], targets)
        #print(f"Training loss: {loss_val.item():.3f}")
        
        #print_batch_accuracy(data, targets, train=True)
        #spk_rec_output, _ = net(data.view(batch_size, -1))
        #_, idx = spk_rec_output.sum(dim=0).max(1)
        #print('spk_rec values and indices',spk_rec_output.sum(dim=0).max(1), 'idx:', idx , 'idx.size',idx.size())
        #acc = np.mean((targets == idx).detach().cpu().numpy())
        #print('acc', acc)
     '''   
# TODO ; diff_data is important, test it and add it
 

In [ ]:
# Plot Loss
fig = plt.figure(facecolor="w", figsize=(10, 5))
plt.plot(loss_hist)
plt.plot(test_loss_hist)
plt.title("Loss Curves")
plt.legend(["Train Loss", "Test Loss"])
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()

# Plot Accuracy
fig = plt.figure(facecolor="w", figsize=(10, 5))
plt.plot(acc_hist)
plt.plot(test_acc_hist)
plt.title("Accuracy Curves")
plt.legend(["Train accuracy", "Test Accuracy"])
plt.xlabel("Iteration")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
############## Tutorial 6 : Surrogate Gradient Descent in a Convolutional Spiking Neural Network for 2 channels
#Solution to dead neuron problem : Smooth the Spike- Heaviside Step function during backward pass to get a smooth gradient.
# This smoothing function is Shifted(Centered at Uth) SIGMOID or FAST-SIGMOID 
# K is Hyperparameter, as K increases, Sigmoid function -> Heaviside Step function

# FORWARD PASS - Determine S using Shifted Heaviside Function
# BACKWARD PASS - Sigmoid. (or Spike Operator)

spike_grad = surrogate.fast_sigmoid(slope=25)
beta = 0.8
num_steps = 50
batch_size_train = 47  
batch_size_test = 94 
# reference self.lif1 = snn.Leaky(beta=beta, threshold=0.1) | snn.Leaky(beta=beta, spike_grad=spike_grad)

# Define Network   : This goes out of memory, seqeunctial does not!
'''
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # Initialize layers
        self.conv1 = nn.Conv2d(1, 15, (2,25))
        self.lif1 = snn.Leaky(beta=beta, threshold=0.1, spike_grad=spike_grad)
        self.conv2 = nn.Conv2d(15, 15, (1,11))
        self.lif2 = snn.Leaky(beta=beta, threshold=0.1, spike_grad=spike_grad)
        self.conv3 = nn.Conv2d(15, 10, (1,5))
        self.lif3 = snn.Leaky(beta=beta, threshold=0.1, spike_grad=spike_grad)
        self.fc1 = nn.Linear(10*1*38, 2)
        self.lif4 = snn.Leaky(beta=beta, threshold=0.1, spike_grad=spike_grad)

    def forward(self, x):

        # Initialize hidden states and outputs at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky() 
        mem3 = self.lif3.init_leaky()
        mem4 = self.lif4.init_leaky()

        # Record the final layer
        spk4_rec = []
        mem4_rec = []

        for step in range(num_steps):
            cur1 = F.max_pool2d(self.conv1(x), (1,4))
            spk1, mem1 = self.lif1(cur1, mem1)
            cur2 = F.max_pool2d(self.conv2(spk1), (1,4))
            spk2, mem2 = self.lif2(cur2, mem2)
            cur3 = F.max_pool2d(self.conv3(spk2), (1,4))
            spk3, mem3 = self.lif3(cur3, mem3)
            cur4 = self.fc1(spk3.view(batch_size_train, -1))
            spk4, mem4 = self.lif4(cur4, mem4)

            spk4_rec.append(spk4)
            mem4_rec.append(mem4)

        return torch.stack(spk4_rec), torch.stack(mem4_rec)
        #'''
net = nn.Sequential(nn.Conv2d(1, 15, (2,25)),
                    snn.Leaky(beta=beta,threshold=1.0, spike_grad=spike_grad, init_hidden=True),
                    nn.MaxPool2d((1,4)),
                    nn.Conv2d(15, 15, (1,11)),
                    #nn.BatchNorm2d(15),
                    snn.Leaky(beta=beta,threshold=1.0, spike_grad=spike_grad, init_hidden=True),
                    nn.MaxPool2d((1,4)),
                    nn.Conv2d(15, 10, (1,5)),
                    #nn.BatchNorm2d(10),
                    snn.Leaky(beta=beta,threshold=1.0, spike_grad=spike_grad, init_hidden=True),
                    nn.MaxPool2d((1,4)),
                    nn.Flatten(),
                    nn.Linear(10*1*38, 2),
                    snn.Leaky(beta=beta, threshold=1.0, spike_grad=spike_grad, init_hidden=True, output=True)
                    ).to(device)


net1 = nn.Sequential(nn.Conv2d(1, 15, (2,25)),
                    nn.MaxPool2d((1,4)),
                    snn.Leaky(beta=beta,threshold=0.7, spike_grad=None, init_hidden=True),
                    nn.Conv2d(15, 15, (1,11)),
                    nn.BatchNorm2d(15),
                    nn.MaxPool2d((1,4)),
                    snn.Leaky(beta=beta,threshold=0.7, spike_grad=None, init_hidden=True),
                    nn.Conv2d(15, 10, (1,5)),
                    nn.BatchNorm2d(10),
                    nn.MaxPool2d((1,4)),
                    snn.Leaky(beta=beta,threshold=0.7, spike_grad=None, init_hidden=True),
                    nn.Flatten(),
                    nn.Linear(10*1*38, 2),
                    snn.Leaky(beta=beta, threshold=0.7, spike_grad=None, init_hidden=True, output=True)
                    ).to(device)#'''
#net = Net().to(device)
print(net)

def print_batch_accuracy(data, targets,batch_size=batch_size_test ,train=False):
    #output, _ = net(data.view(batch_size, -1))
    output, mem_rec = forward_pass(net, num_steps, data)
    check_all_output = output.sum(dim=0)
    print('spk_rec values', check_all_output.size(),'Spike accumulated for both labels', check_all_output )
    val, idx = output.sum(dim=0).max(1)
    print('idx from',output.sum(dim=0).max(1), 'idx.size',idx.size(), 'targets', targets)
    acc = np.mean((targets == idx).detach().cpu().numpy())
    #print('acc', acc)

    if train:
        print(f"Train set accuracy for a single minibatch: {acc*100:.2f}%")
        acc_hist.append(acc.item())
    else:
        print(f"Test set accuracy for a single minibatch: {acc*100:.2f}%")
        test_acc_hist.append(acc.item())

def train_printer():
    print(f"Epoch {epoch}")
    print('Counter', counter)
    print(f"Train Set Loss: {loss_hist[counter]:.2f}")
    print(f"Test Set Loss: {test_loss_hist[counter]:.2f}")
    print_batch_accuracy(data, targets, batch_size=batch_size_train, train=True)
    print_batch_accuracy(test_data, test_targets, batch_size=batch_size_test,train=False)
    print(f"Train Accuracy: {acc_hist[counter]:.2f}")
    print(f"Test Accuracy: {test_acc_hist[counter]:.2f}")
    print("\n")
    
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4, betas=(0.9, 0.999))
loss = SF.ce_rate_loss()
#loss = nn.CrossEntropyLoss()

#Cross Entropy spike rate loss at EVERY TIME STEP -Combines logStoftmax and NLLLoss.
#The losses are accumulated over time steps to give the final loss and encourage firing of correct class.

In [ ]:
# For one sample

def forward_pass(net, num_steps, data):
    mem_rec = []
    spk_rec = []
    utils.reset(net)  # resets hidden states for all LIF neurons in net

    for step in range(num_steps):
        spk_out, mem_out = net(data)
        spk_rec.append(spk_out)
        mem_rec.append(mem_out)
  
    return torch.stack(spk_rec), torch.stack(mem_rec)

data, targets = next(iter(train_loader))   # Normalized data
data = data.to(device=device, dtype=torch.float32)
#indices = torch.tensor([0])
#data = torch.index_select(data, 2, indices)                      # Select only 1 channel
#data = spikegen.delta(data, threshold=0.5)
targets = targets.to(device=device, dtype=torch.int64)
print('data', data.shape)
#spk_rec, mem_rec = net(data.view(batch_size_train, -1))
spk_rec, mem_rec = forward_pass(net, num_steps, data)
print( 'spk_rec',spk_rec, 'mem_rec',mem_rec)
print('targets', targets, 'spk_rec', spk_rec)

# initialize the total loss value
loss_val = torch.zeros((1), dtype=dtype, device=device)
loss_val = loss(spk_rec, targets)

print('loss_val', loss_val, loss_val.size())
print(f"Training loss: {loss_val.item():.3f}")

#print_batch_accuracy(data, targets, train=True)
_, idx = spk_rec.sum(dim=0).max(1)
accuracy = np.mean((targets == idx).detach().cpu().numpy())
acc = SF.accuracy_rate(spk_rec, targets)
print('1st run idx from',spk_rec.sum(dim=0).max(1), 'idx.size',idx.size(), 'targets', targets, 'accuracy', acc)

# clear previously stored gradients
optimizer.zero_grad()

# calculate the gradients
loss_val.backward()

# weight update
optimizer.step()

# calculate new network outputs using the same data
#spk_rec, mem_rec = net(data.view(batch_size_train, -1))
spk_rec, mem_rec = forward_pass(net, num_steps, data)
print( 'spk_rec',spk_rec, 'mem_rec',mem_rec)
print('targets', targets, 'spk_rec', spk_rec)

# initialize the total loss value
loss_val = torch.zeros((1), dtype=dtype, device=device)
    
loss_val = loss(spk_rec, targets)

print(f"Training loss: {loss_val.item():.3f}")
#print_batch_accuracy(data, targets, train=True)#''' # end of one sample
_, idx = spk_rec.sum(dim=0).max(1)
accuracy = np.mean((targets == idx).detach().cpu().numpy())
acc = SF.accuracy_rate(spk_rec, targets)
print('2nd run idx from',spk_rec.sum(dim=0).max(1), 'idx.size',idx.size(), 'targets', targets, 'accuracy', acc)

In [ ]:
#Training on entire dataset

num_epochs = 1
loss_hist = []
test_loss_hist = []
acc_hist = []
test_acc_hist = []
counter = 0
indices = torch.tensor([0])

def forward_pass(net, num_steps, data):
    mem_rec = []
    spk_rec = []
    utils.reset(net)  # resets hidden states for all LIF neurons in net

    for step in range(num_steps):
        spk_out, mem_out = net(data)
        spk_rec.append(spk_out)
        mem_rec.append(mem_out)
  
    return torch.stack(spk_rec), torch.stack(mem_rec)

optimizer = torch.optim.Adam(net.parameters(), lr=5e-4, betas=(0.9, 0.999))
loss = SF.ce_rate_loss()

# Outer training loop
for epoch in range(num_epochs):
    iter_counter = 0
    
    # Minibatch training loop
    for data, targets in iter(train_loader):
        data = data.to(device=device, dtype=torch.float32)
        #data = torch.index_select(data, 2, indices)
        print('data', data.shape, data)
        check_nan = torch.isnan(data)
        #print('idx for nan in training',tf.where(check_nan))
        print('does input have nan training', torch.count_nonzero(check_nan))
        if (torch.count_nonzero(check_nan) != 0):
            corrupted_batch = (tf.where(check_nan))[0][0]
            #print('corrupted data', data.shape,'corrupt batch',corrupted_batch)
            for i in range(0,47):
                print('check if data is corrupt',i,data[i])
        #print('data',data.size(), data)
        #data = spikegen.delta(data, threshold=0.5)
        #print('spikegen data',data.size(), data)
        targets = targets.to(device=device, dtype=torch.int64)
        #print('targets',targets.size(), targets)
        
        print('######## DURING TRAINING ############')
        # forward pass
        net.train()
        #print('data',data.shape, 'input data shape', data.view(batch_size_train, -1).shape)
        #spk_rec, mem_rec = net(data.view(batch_size_train, -1))
        spk_rec, mem_rec = forward_pass(net, num_steps, data)
        print( 'spk_rec',spk_rec, 'mem_rec',mem_rec)
        print('targets', targets, 'spk_rec', spk_rec)
        
        #print('spk_rec',spk_rec.size(), 'mem_rec',mem_rec.size())
        #print('spk_rec',spk_rec, 'mem_rec',mem_rec)
        #for name, param in net.named_parameters():
        #    if param.requires_grad:
        #        print('Name of parameter',name, 'Parameter data', (param.data).shape,param.data)
        #print('Non-zero elements in spk_rec',torch.count_nonzero(spk_rec))
        
        # initialize the loss & sum over time
        loss_val = torch.zeros((1), dtype=dtype, device=device)
        for step in range(num_steps):
            #print('Check loss dims', num_steps, mem_rec[step].shape, targets.shape)
            #loss_val += loss(mem_rec[step], targets)   # Used loss for FCN
            loss_val += loss(spk_rec, targets)
        #print('Loss val for train ', loss_val)
        #loss_val = loss(spk_rec, targets)

        # Gradient calculation + weight update
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

        # Store loss history for future plotting
        loss_hist.append(loss_val.item())
        
        #train_printer()
        
        # Test set
        with torch.no_grad():
            net.eval()
            test_data, test_targets = next(iter(test_loader))
            test_data = test_data.to(device=device, dtype=torch.float32)
            #test_data = torch.index_select(test_data, 2, indices)
            print('test_data', test_data.shape, test_data)
            check_nan = torch.isnan(test_data)
            print('idx for nan in testing',tf.where(check_nan))
            print('does input have nan testing', torch.count_nonzero(check_nan))
            #test_data = spikegen.delta(test_data, threshold=0.5)
            test_targets = test_targets.to(device=device, dtype=torch.int64)
            
            print('######## DURING TESTING ############')
            # Test set forward pass
            #print('test_data',test_data.shape, 'input test_data', test_data.view(batch_size_test, -1).shape)
            #test_spk, test_mem = net(test_data.view(batch_size_test, -1))
            test_spk, test_mem = forward_pass(net, num_steps, test_data)
            #print('spk_rec',test_spk.size(), 'mem_rec',test_mem.size())
            #print('spk_rec',test_spk, 'mem_rec',test_mem)
            #for name, param in net.named_parameters():
            #    if param.requires_grad:
            #        print('Name of parameter',name, 'Parameter data', (param.data).shape,param.data)
            
            # Test set loss
            test_loss = torch.zeros((1), dtype=dtype, device=device)
            for step in range(num_steps):
                test_loss += loss(test_spk, test_targets)
            #print('Loss val for test ', test_loss)
            test_loss_hist.append(test_loss.item())

            # Print train/test loss/accuracy
            #if counter % 10 == 0:
            train_printer()
            counter += 1
            #iter_counter +=1
            #print('counter', counter, 'iter_counter', iter_counter)
        print('####### END OF AN ITERATION')

In [ ]:
# Plot Loss
fig = plt.figure(facecolor="w", figsize=(10, 5))
plt.plot(loss_hist)
plt.plot(test_loss_hist)
plt.title("Loss Curves")
plt.legend(["Train Loss", "Test Loss"])
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()

# Plot Accuracy
fig = plt.figure(facecolor="w", figsize=(10, 5))
plt.plot(acc_hist)
plt.plot(test_acc_hist)
plt.title("Accuracy Curves")
plt.legend(["Train accuracy", "Test Accuracy"])
plt.xlabel("Iteration")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
# this time, we won't return membrane as we don't need it 
def forward_pass(net, data):  
    spk_rec = []
    mem_rec =[]
    utils.reset(net)  # resets hidden states for all LIF neurons in net

    #for step in range(data.size(0)):  # data.size(0) = number of time steps
    #    spk_out, mem_out = net(data[step])
    for step in range(num_steps):  # data.size(0) = number of time steps
        spk_out, mem_out = net(data)
        mem_rec.append(mem_out)
        spk_rec.append(spk_out)
  
    return torch.stack(spk_rec), torch.stack(mem_rec)

optimizer = torch.optim.Adam(net.parameters(), lr=2e-2, betas=(0.9, 0.999))
loss_fn = SF.mse_count_loss(correct_rate=0.8, incorrect_rate=0.2)
#loss_fn = SF.ce_rate_loss()


num_epochs = 3
num_iters = 50

loss_hist = []
acc_hist = []

# training loop
for epoch in range(num_epochs):
    for i, (data, targets) in enumerate(iter(train_loader)):
        data = data.to(device=device, dtype=torch.float32)
        targets = targets.to(device=device, dtype=torch.int64)

        net.train()
        spk_rec, mem_rec = forward_pass(net, data)
        #for name, param in net.named_parameters():
        #    if param.requires_grad:
        #        print('Name of parameter',name, 'Parameter data', (param.data).shape,param.data)
        print('mem_rec', spk_rec, mem_rec, 'Truth', targets)
        loss_val = loss_fn(spk_rec, targets)

        # Gradient calculation + weight update
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

        # Store loss history for future plotting
        loss_hist.append(loss_val.item())
 
        print(f"Epoch {epoch}, Iteration {i} \nTrain Loss: {loss_val.item():.2f}")

        acc = SF.accuracy_rate(spk_rec, targets) 
        acc_hist.append(acc)
        print(f"Accuracy: {acc * 100:.2f}%\n")

        # This will end training after 50 iterations by default
        if i == num_iters:
            break

In [ ]:
#!pip install torchsummary
# neuron and simulation parameters
spike_grad = surrogate.fast_sigmoid(slope=25)
beta = 0.5
num_steps = 50
#device = 'cuda'

from torchsummary import summary
from torch import nn
import torch.nn.functional as F

print(device)

net = nn.Sequential(nn.Conv2d(1, 15, (2,25)),
                    nn.MaxPool2d((1,4)),
                    snn.Leaky(beta=beta,threshold=1.0, spike_grad=spike_grad, init_hidden=True),
                    nn.Conv2d(15, 15, (1,11)),
                    nn.BatchNorm2d(15),
                    nn.MaxPool2d((1,4)),
                    snn.Leaky(beta=beta,threshold=1.0, spike_grad=spike_grad, init_hidden=True),
                    nn.Conv2d(15, 10, (1,5)),
                    nn.BatchNorm2d(10),
                    nn.MaxPool2d((1,4)),
                    snn.Leaky(beta=beta,threshold=1.0, spike_grad=spike_grad, init_hidden=True),
                    nn.Flatten(),
                    nn.Linear(10*1*38, 2),
                    snn.Leaky(beta=beta, threshold=1.0, spike_grad=spike_grad, init_hidden=True, output=True)
                    ).to(device)

#summary(net.to(device), ( 1, 2, 2560), 180)
print(net)

In [ ]:
# Plot Loss
fig = plt.figure(facecolor="w", figsize=(10, 5))
plt.plot(loss_hist)
plt.plot(test_loss_hist)
plt.title("Loss Curves")
plt.legend(["Train Loss", "Test Loss"])
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()

total = 0
correct = 0

# drop_last switched to False to keep all samples
#test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=True, drop_last=False)

with torch.no_grad():
    net.eval()
    for name, param in net.named_parameters():
        if param.requires_grad:
            print(name, param.data) 
    for data, targets in test_loader:
        data = data.to(device=device, dtype=torch.float32)
        targets = targets.to(device=device, dtype=torch.int64)
    
        # forward pass
        test_spk, _ = net(data.view(data.size(0), -1))

        # calculate total accuracy
        _, predicted = test_spk.sum(dim=0).max(1)
        total += targets.size(0)
        print('Targets', targets, 'Prediction', predicted)
        correct += (predicted == targets).sum().item()

print(f"Total correctly classified test set images: {correct}/{total}")
print(f"Test Set Accuracy: {100 * correct / total:.2f}%")


In [ ]:
#Experiments with Data Encoding
from snntorch import spikegen
from snntorch import utils 
import matplotlib.pyplot as plt
import snntorch.spikeplot as splt
from IPython.display import HTML
num_steps = 25

for i, (data, targets) in enumerate(iter(test_loader)):
        data = data.to(device=device, dtype=torch.float32)
        print('data',  data.size())
        targets = targets.to(device=device, dtype=torch.int64)
        print('targets', targets.size())
        '''
        #Delta Modulation
        spike_data = spikegen.delta(data, threshold=0.1)

        # Create fig, ax
        fig = plt.figure(facecolor="w", figsize=(8, 1))
        ax = fig.add_subplot(111)

        # Raster plot of delta converted data
        splt.raster(spike_data[0,0,0,:], ax)

        plt.title("Input Neuron")
        plt.xlabel("Time step")
        plt.yticks([])
        #plt.xlim(0, len(data))
        plt.show()
        '''
        
        #Rate Encode
        spike_data_delta = spikegen.delta(data, threshold=1.0)
        spike_data = spikegen.rate(spike_data_delta, num_steps=20)
        print('spike_data', spike_data.size())
        
        spike_data_sample = spike_data[:, 84, 0]
        #print(spike_data_sample.size())
        #fig, ax = plt.subplots()
        #anim = splt.animator(spike_data_sample, fig, ax)
        #HTML(anim.to_html5_video())
        
        spike_data_sample2 = spike_data_sample.reshape((num_steps, -1))
        print('targets',targets, targets.size())
        print('spike_data_sample2', spike_data_sample2, spike_data_sample2.size())

        # raster plot
        fig = plt.figure(facecolor="w", figsize=(10, 5))
        ax = fig.add_subplot(111)
        splt.raster(spike_data_sample2, ax, s=1.5, c="black")

        plt.title("Input Layer")
        plt.xlabel("Time step")
        plt.ylabel("Neuron Number")
        plt.show()
        

In [ ]:
#Training SNN 1
from snntorch import utils 
from sklearn.metrics import balanced_accuracy_score
from IPython.display import HTML

def analog_to_spike(batch_x, batch_y):
    #print('shape for analog to spike',np.shape(batch_x), np.shape(batch_x)[0])
    b_s = np.shape(batch_x)[0]
    c_s = np.shape(batch_x)[2]
    t_s = np.shape(batch_x)[3]
    
    #batch_x = spikegen.delta(batch_x, threshold=0.4, padding=True, off_spike=False) # Better visualization at 0.4 than 0.5
    batch_x = spikegen.rate(batch_x, num_steps=2 ,gain=1, offset=0, first_spike_time=0, time_var_input=False )
    '''
    for b in range(0,1):
        for c in range(0,c_s):    
            # Create fig, ax
            fig = plt.figure(facecolor="w", figsize=(8, 4))
            ax = fig.add_subplot(111)

            # Raster plot of delta converted data
            splt.raster(batch_x[b,0,c,:], ax)

            plt.title("Input Neuron "+ "for batchsample_"+ str(b) +"_for channel_"+ str(c) + "_with_label:" +str(batch_y[b]))
            plt.xlabel("Time step")
            plt.yticks([])
            plt.xlim(0,t_s )
            fig.canvas.draw()
        break #'''
   
    return batch_x

def forward_pass(net, data, num_steps):  
    spk_rec = []
    mem_rec = []
    utils.reset(net)  # resets hidden states for all LIF neurons in net

    for step in range(num_steps): 
        spk_out, mem_out = net(data)
        spk_rec.append(spk_out)
        mem_rec.append(mem_out)
    #print('spk_rec',torch.stack(spk_rec).size(), 'mem_rec',torch.stack(mem_rec).size()) [10, 180, 2]
    return torch.stack(spk_rec), torch.stack(mem_rec)

import snntorch.functional as SF

optimizer = torch.optim.Adam(net.parameters(), lr=2e-3, betas=(0.9, 0.999))
#loss_fn = SF.mse_count_loss(correct_rate=0.8, incorrect_rate=0.2)
loss_fn = SF.ce_max_membrane_loss()


num_epochs = 1
num_steps = 10  # run for 25 time steps 

loss_hist = []
acc_hist = []
#train_loss = np.zeros(len(loader_train))
#y_pred_all, y_true_all = list(), list()
#metric = balanced_accuracy_score

# Testing loop
def test_accuracy(data_loader, net, num_steps):
    with torch.no_grad():
        total = 0
        acc = 0
        net.eval()
        #for name, param in net.named_parameters():
        #    if param.requires_grad:
        #        print( name, param.data)
                
        data_loader = iter(data_loader)
        for data, targets in data_loader:
            data = data.to(device=device, dtype=torch.float32)
            targets = targets.to(device=device, dtype=torch.int64)
            data = analog_to_spike(data,targets)
            spk_rec, _ = forward_pass(net, data, num_steps)  #spk_rec is [10, 256, 2]
            #print('Test accuracy vals', spk_rec.size(), 'targets',targets.size(), 'size',spk_rec.size(1))
            _, idx = spk_rec.sum(dim=0).max(1)
            #print('target test', targets, 'idx test', idx)
            accuracy = np.mean((targets == idx).detach().cpu().numpy())
            #print('Decoded accuracy', accuracy)
            acc += SF.accuracy_rate(spk_rec, targets) * spk_rec.size(1)
            total += spk_rec.size(1)
    print('Accuracy', acc, 'Total', total)
    #print(f"Accuracy testing: {acc/total:.2f}%")

    return acc/total
# TODO : data (torch.Tensor) – Data tensor for a single batch of shape [num_steps x batch x input_size]
# or use Rate data (torch.Tensor) – Data tensor for a single batch of shape [batch x input_size]x
# training loop
for epoch in range(num_epochs):
    for i, (data, targets) in enumerate(iter(loader_train)):
        data = data.to(device=device, dtype=torch.float32)
        targets = targets.to(device=device, dtype=torch.int64)
        data = analog_to_spike(data,targets)
        
        net.train()
        spk_rec, _ = forward_pass(net, data, num_steps)
        
        loss_val = loss_fn(spk_rec, targets)

        # Gradient calculation + weight update
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()
        
        #y_pred_all.append(torch.argmax(spk_rec, axis=1).cpu().numpy())
        #y_true_all.append(targets.cpu().numpy())
        
        # Store loss history for future plotting
        loss_hist.append(loss_val.item())
        #train_loss[i] = loss_val.item()

        # print every 25 iterations
        if i % 25 == 0:
            print(f"Epoch {epoch}, Iteration {i} \nTrain Loss: {loss_val.item():.2f}")
            _, idx = spk_rec.sum(dim=0).max(1)
            print('target train', targets, 'idx train', idx)
            # check accuracy on a single batch
            acc = SF.accuracy_rate(spk_rec, targets)  
            acc_hist.append(acc)
            #print(f"Accuracy training: {acc * 100:.2f}")
            
    #print(net)        
    print(f"Test set accuracy epoch: {test_accuracy(loader_test, net, num_steps)*100:.3f}%")     
    
    #y_pred = np.concatenate(y_pred_all)
    #y_true = np.concatenate(y_true_all)
    #perf = metric(y_true, y_pred)
    #print('Perf',np.mean(train_loss) , 'y_pred',y_pred,'y_true', y_true,'len', np.shape(y_pred))
    #print('loss_hist', loss_hist, len(loss_hist))
    print('End of Epoch\n')
'''
#With time_var = True

from snntorch import backprop

num_epochs = 3
#print('Num_steps is defined?', num_steps)
# training loop
for epoch in range(num_epochs):
    for data, target in loader_train:
        data = data.to(device=device, dtype=torch.float32)
        target = target.to(device=device, dtype=torch.int64)
        data = analog_to_spike(data,target)
        print(data.size(), target.size())
        loss = backprop.RTRL(net, data, target, optimizer, loss_fn, num_steps=False, time_var=True, device=device)

    print(f"Epoch {epoch}, Train Loss: {loss.item():.2f}")
'''

#print(f"Test set accuracy: {test_accuracy(loader_test, net, num_steps)*100:.3f}%")


In [ ]:
#Plot statistics for SNN1

# Visualizing the learning curves
history_df = pd.DataFrame(history)
ax1 = history_df.plot(x='epoch', y=['train_loss', 'valid_loss'], marker='o')
ax1.set_ylabel('Loss')
ax2 = history_df.plot(x='epoch', y=['train_perf', 'valid_perf'], marker='o')
ax2.set_ylabel('balanced_accuracy_score')

# VISUALIZE THE RESULTS
classes_mapping = {0: 'non-ictal', 1: 'ictal'}
y_train = pd.Series([y for _, y in train_ds]).map(classes_mapping)
#Visualizing the class distribution, Very imbalanced right now #TODO
#ax_ = y_train.value_counts().plot(kind='barh')
#ax_.set_xlabel('Number of training examples');
#ax_.set_ylabel('Sleep stage');


from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(conf_mat, classes_mapping):
    ticks = list(classes_mapping.keys())
    tick_labels = classes_mapping.values()

    fig, ax = plt.subplots(figsize=(6, 6))
    im = ax.imshow(conf_mat, cmap='Reds')

    ax.set_yticks(ticks)
    ax.set_yticklabels(tick_labels)
    ax.set_xticks(ticks)
    ax.set_xticklabels(tick_labels)
    ax.set_ylabel('True label')
    ax.set_xlabel('Predicted label')
    ax.set_title('Confusion matrix')

    for i in range(len(ticks)):
        for j in range(len(ticks)):
            text = ax.text(
                j, i, conf_mat[i, j], ha='center', va='center', color='k')

    fig.colorbar(im, ax=ax, fraction=0.05, label='# examples')
    fig.tight_layout()
    
    return fig, ax

conf_mat = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(conf_mat, classes_mapping);


In [ ]:
# TEMP SNN coding suggestions here 
'''
from snntorch import surrogate
import torch
import torch.nn as nn

beta = 0.9

# Initialize surrogate gradient
spike_grad = surrogate.fast_sigmoid()  # passes default parameters from a closure

# Define Network
class Net(nn.Module):
 def __init__(self):
     super().__init__()

 # Initialize layers, specify the ``spike_grad`` argument
     self.fc1 = nn.Linear(num_inputs, num_hidden)
     self.lif1 = snn.Leaky(beta=beta, spike_grad=spike_grad)
     self.fc2 = nn.Linear(num_hidden, num_outputs)
     self.lif2 = snn.Leaky(beta=beta, spike_grad=spike_grad)
         lif1 = snn.Leaky(beta=0.9, init_hidden=True) # only returns spk
         lif2 = snn.Leaky(beta=0.9, init_hidden=True, output=True) # returns mem and spk if output=True

When initializing neurons, set init_hidden=True. This enables the methods in snntorch.
backprop to automatically clear the hidden state variables, as well as detach them from the computational graph when necessary.
 
 def forward(self, x, syn1, mem1, spk1, syn2, mem2):
     cur1 = self.fc1(x)
     spk1, mem1 = self.lif1(cur1, mem1)
     cur2 = self.fc2(spk1)
     spk2, mem2 = self.lif2(cur2, mem2)
     return mem1, spk1, mem2, spk2

 net = Net().to(device)

optimizer = torch.optim.Adam(net.parameters(), lr=lr, betas=betas)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(net.parameters(), lr=2e-3, betas=(0.9, 0.999))
loss_fn = SF.mse_count_loss(correct_rate=0.8, incorrect_rate=0.2)

# Time-variant input data
for input, target in dataset:
   loss = BPTT(net, input, target, num_steps, batch_size, optimizer, criterion)  # THIS IS WHAT I SHOULD USE!

# Time-invariant input data
for input, targets in dataset:
   loss = BPTT(net, input, target, num_steps, batch_size, optimizer, criterion, time_varying_data=False)


###########################################################################
For making alpha and beta a learnable parameter for each layer than hyperparameter

The same approach as above can be used for implementing learnable thresholds, using learn_threshold=True.

Each neuron has the option to inhibit other neurons within the same dense layer from firing. 
This can be invoked by setting inhibition=True. Only with FC layer

         self.lif1 = snn.Synaptic(alpha=alpha, beta=beta,
                                 spike_grad=spike_grad,
                                 threshold=0.5, learn_alpha=True,
                                 learn_beta=True)
        self.lif2 = snn.Synaptic(alpha=alpha, beta=beta,
                                 spike_grad=spike_grad,
                                 threshold=0.5, learn_alpha=True,
                                 learn_beta=True)
        self.lif3 = snn.Synaptic(alpha=alpha, beta=beta,
                                 spike_grad=spike_grad,
                                 threshold=0.5, learn_alpha=True,
                                 learn_beta=True)
        self.lif4 = snn.Synaptic(alpha=alpha, beta=beta,
                                 spike_grad=spike_grad,
                                 threshold=0.5, learn_alpha=True,
                                 learn_beta=True)
        self.lif5 = snn.Synaptic(alpha=alpha, beta=beta,
                                 spike_grad=spike_grad,
                                 threshold=0.5, learn_alpha=True,
                                 learn_beta=True)
############################################################################

beta = 0.9
a = 1

threshold = nn.Parameter( a )
lif = snn.Leaky(beta, threshold=threshold)

. Alternatively, a = torch.rand((num_outputs), dtype=torch.float32) will let you set individual thresholds per neuron.

self.register_parameter(name = "threshold", param = threshold)
############################################################################
NOTE TOD: Try out different type of backprops 
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4, betas=(0.9, 0.999))
loss_fn = SF.mse_count_loss()
reg_fn = SF.l1_rate_sparsity()


# train_loader is of type torch.utils.data.DataLoader
# if input data is time-static, set time_var=False, and specify num_steps.
# if input data is time-varying, set time_var=True and do not specify num_steps. (only for backprop)

for epoch in range(5):
    loss = backprop.RTRL(net, train_loader, optimizer=optimizer,
    criterion=loss_fn, num_steps=num_steps, time_var=False,
    regularization=reg_fn, device=device)
    
############################################################################


'''